In [ ]:
# configuration and imports
data_folder = "/data/benchmark/log/"
output_folder = "/data/benchmark/out/"

workflow = "workflowB"
computerName = "bwb"

import utils
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

font = {'family' : 'normal',
        'size'   : 22}

matplotlib.rc('font', **font)

In [ ]:
# load data

# the dataset contains 150 measurements of cell count in Drosophila melanogaster imaging data
# Furthermore, measurements exist for Laptop versus Workstation and ImageJ versus CLIJ

cell_count_clij = utils.read_all_measurements_from_file(data_folder + computerName + "_" + workflow + "_cellcount_clij.csv")
cell_count_imagej = utils.read_all_measurements_from_file(data_folder + computerName + "_" + workflow + "_cellcount_imagej.csv")

In [ ]:
# plot number of spots over time for each sample
timepoints = range(0, len(cell_count_imagej))
plt.figure(figsize=(10, 10), dpi=80)
plt.plot(timepoints, cell_count_imagej, '.')
plt.plot(timepoints, cell_count_clij, '.')

plt.legend(["ImageJ", "CLIJ"])
plt.xlabel("Time point")
plt.ylabel("Spot count")
plt.title("Spot count over time")
plt.savefig(output_folder + "spot_count_plot.png")
plt.show()

In [ ]:
# some more definitions 
def compare(data1, data2, data1_name, data2_name):
    comparison_name = data1_name + "_versus_" + data2_name

    # Bland Altman analysis
    # https://www-users.york.ac.uk/~mb55/meas/ba.pdf

    utils.bland_altman_plot(data1, data2)

    plt.title('Bland-Altman Plot of spot count')
    plt.xlabel("Average spot count")
    plt.ylabel("Spot count " + data1_name + " - spot count " + data2_name)

    plt.savefig(output_folder + "spot_count_" + comparison_name + "_bland_altman_plot.png")
    plt.show()

    # coefficient of repeatability (Bland Altman 1973)
    squared_differences = np.power(np.subtract(data1, data2), 2)


    CR = np.sqrt(np.sum(squared_differences) / len(squared_differences))

    print("Bland Altmans coefficient of repeatability: " + str(CR))

    ############################################################################3
    # Pearson correlation analysis

    plt.figure(figsize=(10, 10), dpi=80)
    plt.plot(data1, data2, 'o')
    plt.xlabel("Spot count " + data1_name)
    plt.ylabel("Spot count " + data2_name)
    plt.title("Spot count scatter plot")
    plt.savefig(output_folder + "spot_count_" + comparison_name + "_scatter_plot.png")
    plt.show()

    # Pearsons correlation coefficient

    pcc = stats.pearsonr(data1, data2)

    print("Pearsons correlation coeficient: " + str(pcc[0]))

    #########################################################################
    # equivalence testing
    # check that relative difference is smaller than a given bound
    max_difference_bounds = 0.01

    differences = np.subtract(data1, data2)
    averages = np.divide(np.add(data1, data2), 2)

    relative_difference = np.abs(np.divide(differences, averages))
    print("Mean relative difference " + str(np.mean(relative_difference)))
    print("StdDev relative difference " + str(np.std(relative_difference)))

    t1, pv = stats.ttest_1samp(relative_difference, max_difference_bounds)

    print("Null-hypothesis: relative difference (" + comparison_name + ") is larger than " + str(max_difference_bounds * 100) + "%")
    print("alternative: relative difference (" + comparison_name + ") is smaller than "  + str(max_difference_bounds * 100) + "%")
    print("p value: " + str(pv))

In [ ]:
print("\nImageJ versus CLIJ")
print("------------------")
compare(cell_count_imagej, cell_count_clij, "ImageJ", "CLIJ")

In [ ]:
#   # generate changing plots over time
#   intended_figure_width = 1250
#   intended_figure_height = 350
#   intended_dpi = 60


#   font = {'family' : 'normal',
#           'size'   : 18}

#   matplotlib.rc('font', **font)

#   for t in range(0, len(cell_count_imagej_laptop)):
#   #for t in range(0, 3):
#       timepoints = range(0, len(cell_count_imagej_laptop[:t]))
#       fig = plt.figure(figsize=(intended_figure_width / intended_dpi, intended_figure_height / intended_dpi), dpi=intended_dpi)
#       plt.xlim(0, 300)
#       plt.ylim(1500, 3500)
    
#       plt.plot(timepoints, cell_count_imagej[:t], 'o')
#       plt.plot(timepoints, cell_count_clij[:t], '.')
    
#       plt.legend(["ImageJ laptop", "CLIJ laptop", "ImageJ workstation", "CLIJ workstation"], loc='lower left') #, "CLIJ Workstation"])
#       plt.xlabel("Time point")
#       plt.ylabel("Spot count")
#       #plt.title("Spot count over time")
    
#       number = "00000" + str(t);
#       print(number[-5:])
#       plt.savefig("../images/spotcount_overtime/spot_count_plot_" + number[-5:] + ".png")
#       # plt.show()